In [16]:
%matplotlib qt5
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

In [50]:
def method1(img,angle=120,rho=141,greyThresh=11,houghThresh=5,lineGap=4,lineLength=14,kernelSize=(5,5),show_helpers=False):
    copy = np.copy(img)
    tmp = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)

    # get current positions of four trackbars
    n,m,_ = tmp.shape
    filtered = np.zeros((n,m,3))

    ret, grey = cv2.threshold(copy,greyThresh,255,cv2.THRESH_BINARY)

    kernel = np.ones(kernelSize,np.uint8)
    closing = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,kernel, iterations = 2)

    ret, helper = cv2.threshold(grey,greyThresh,255,cv2.THRESH_BINARY)
    ret, helper3 = cv2.threshold(closing,greyThresh,255,cv2.THRESH_BINARY)
    hlines = helper3
    ang = angle * np.pi/180

    lines = cv2.HoughLinesP(hlines,rho,ang,houghThresh,lineLength,lineGap)
    for x in range(0, len(lines)):
        for x1,y1,x2,y2 in lines[x]:
            cv2.line(tmp,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.line(filtered,(x1,y1),(x2,y2),(255,255,255),2)

    if(show_helpers):
        cv2.imshow('threshold grey',grey)
        cv2.imshow('closed',closing)
        cv2.imshow('threshold',helper)
        cv2.imshow('threshold closed',helper3)

    cv2.imshow('Probablistic Lines',tmp)
    cv2.imshow('Probablistic Filtered',filtered)
    return filtered

In [58]:
def uvMapping(_img, get_overlay=True,verbose=False):
    overlay = None
    h,w = _img.shape
    histRange = (0,256)
    histSz = np.max(_img) + 1
    if(verbose): print("[UV Mapping] Input Image Size: (%d, %d)" % (h,w))

    umap = np.zeros((histSz,w,1), dtype=np.uint8)
    vmap = np.zeros((h,histSz,1), dtype=np.uint8)

    for i in range(0,h):
        vscan = _img[i,:]
        vrow = cv2.calcHist([vscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[V Mapping] Scan [%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, vscan.shape)), ', '.join(map(str, vrow.shape))))
        vmap[i,:] = vrow

    for i in range(0,w):
        uscan = _img[:,i]
        urow = cv2.calcHist([uscan],[0],None,[histSz],histRange)
        if(verbose): print("\t[U Mapping] Scan[%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, uscan.shape)), ', '.join(map(str, urow.shape))))
        umap[:,i] = urow

    umap = np.reshape(umap,(histSz,w))
    vmap = np.reshape(vmap,(h,histSz))

    if(get_overlay):
        blank = np.ones((umap.shape[0],vmap.shape[1]),np.uint8)*255
        pt1 = np.concatenate((_img, vmap), axis=1)
        pt2 = np.concatenate((umap,blank),axis=1)
        overlay = np.concatenate((pt1,pt2),axis=0)

    if(verbose):
        print("\t[UV Mapping] U Map = (%s) ----- V Map = (%s)" % (', '.join(map(str, umap.shape)),', '.join(map(str, vmap.shape)) ))
    return umap,vmap,overlay

In [60]:
# plt.ion()
imgname = "frames/frame_509.jpg"
img = cv2.imread(imgname,cv2.IMREAD_GRAYSCALE)
umap,vmap, overlay = uvMapping(img)
overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)
plt.imshow(overlay)
plt.yticks([''])
plt.xticks([''])
plt.show()

In [53]:
n1,m1 = umap.shape
n2,m2 = vmap.shape
blank = np.ones((n1,m2),np.uint8)*255

plt.figure(figsize=(8,8))

cd = np.concatenate((img, vmap), axis=1)
cd2 = np.concatenate((umap,blank),axis=1)
overlay = np.concatenate((cd,cd2),axis=0)
overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)

plt.imshow(overlay)
plt.yticks([''])
plt.xticks([''])
# ax.set_yticklabels('')

plt.show()

In [57]:
vscan = np.zeros((26, 22, 2))
vrow = np.zeros((1, 2, 3))

print("\t[V Mapping] V Scan (%s) ---- Scan Histogram (%s)" % (', '.join(map(str, vscan.shape)), ', '.join(map(str, vrow.shape))))

	[V Mapping] V Scan (26, 22, 2) ---- Scan Histogram (1, 2, 3)
